# Pingo / Moodle XML → Q&A PDF

This notebook lets you:

1. Upload your existing Pingo/Moodle XML file (with `<quiz>` and `<question>` blocks).
2. Parse all multiple-choice questions and their correct answers.
3. Generate a nicely formatted PDF:
   - First part: questions with answer options (A, B, C, …)
   - Second part: answer key.


In [16]:
# 📚 2. Imports

import xml.etree.ElementTree as ET
from fpdf import FPDF

# If you use Google Colab, you can use this for file upload/download:
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False


In [39]:
# 📁 3. Upload your XML file

xml_path = "/Users/gollapsi/Documents/17_Hof_Lecture_Code_Pingo/Supply_Chain_Analytics/pingo/Prescriptive_Analytics.xml"  # default name; change if needed

if IN_COLAB:
    print("Please upload your Pingo/Moodle XML file...")
    uploaded = files.upload()
    # Take the first uploaded file
    xml_path = list(uploaded.keys())[0]
    print(f"Using file: {xml_path}")
else:
    # If running locally, put your XML file in the same folder and
    # set xml_path above to its filename.
    print(f"Using local XML file: {xml_path}")


Using local XML file: /Users/gollapsi/Documents/17_Hof_Lecture_Code_Pingo/Supply_Chain_Analytics/pingo/Prescriptive_Analytics.xml


In [40]:
# 🧠 4. Parse XML → Python objects

def parse_moodle_multichoice(xml_file):
    """
    Parses a Moodle-style / Pingo-style XML file with:
    <quiz>
      <question type="multichoice">...</question>
      ...
    </quiz>

    Returns:
        questions: list of dicts with:
            {
                "name": str,
                "question": str,
                "answers": [str, ...],
                "correct_index": int,
            }
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()

    questions = []

    for q in root.findall("question"):
        qtype = q.attrib.get("type", "")

        # Skip category/description/etc; keep only multichoice
        if qtype != "multichoice":
            continue

        # Name
        name_node = q.find("./name/text")
        name = name_node.text.strip() if name_node is not None and name_node.text else ""

        # Question text
        qt_node = q.find("./questiontext/text")
        qtext = qt_node.text.strip() if qt_node is not None and qt_node.text else ""

        answers = []
        correct_index = None

        # Each <answer fraction="100"> is the correct one
        for idx, ans_node in enumerate(q.findall("answer")):
            text_node = ans_node.find("text")
            ans_text = text_node.text.strip() if text_node is not None and text_node.text else ""
            fraction = ans_node.attrib.get("fraction", "0")

            answers.append(ans_text)
            if fraction == "100":
                correct_index = idx

        # Only keep questions that have at least one answer
        if answers:
            questions.append({
                "name": name,
                "question": qtext,
                "answers": answers,
                "correct_index": correct_index
            })

    return questions


questions = parse_moodle_multichoice(xml_path)
print(f"Parsed {len(questions)} multichoice questions.")
if questions:
    print("Example:")
    print(questions[0])


Parsed 5 multichoice questions.
Example:
{'name': 'Q16. DC_East capacity cut', 'question': 'Reducing DC_East capacity from 600 to 500 affects total cost how?', 'answers': ['Cost decreases', 'Cost increases moderately', 'No change', 'Cost drops sharply'], 'correct_index': 1}


In [41]:
from fpdf import FPDF

def _safe(text: str) -> str:
    if text is None:
        return ""
    return text.encode("latin-1", "ignore").decode("latin-1")


def make_pdf_q_and_answer_inline(
    questions,
    pdf_filename="pingo_questions_inline_answers.pdf",
    title="AI in Supply Chain - Pingo Questions"
):
    """
    Layout per question:

    Q1. Question text...
      A. option text...
      B. option text...
      C. option text...
      D. option text...
    Correct answer: A. option text...
    """

    if not questions:
        raise ValueError("No questions to write into PDF.")

    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)

    # Margins
    pdf.set_left_margin(15)
    pdf.set_right_margin(15)

    pdf.add_page()
    effective_width = pdf.w - pdf.l_margin - pdf.r_margin

    # Title
    pdf.set_font("Helvetica", "B", 16)
    pdf.multi_cell(effective_width, 10, _safe(title), align="C")
    pdf.ln(8)

    option_labels = ["A", "B", "C", "D", "E", "F"]
    label_width = 8  # width for "A." cell

    for i, q in enumerate(questions, start=1):
        # Question text
        pdf.set_font("Helvetica", "B", 12)
        q_header = f"Q{i}. {q['question']}"
        pdf.multi_cell(effective_width, 8, _safe(q_header))
        pdf.ln(2)

        # Options: label cell + text multi_cell
        pdf.set_font("Helvetica", "", 11)
        for j, ans_text in enumerate(q["answers"]):
            label = option_labels[j] if j < len(option_labels) else f"{j+1}"
            # Start at left margin each time
            pdf.set_x(pdf.l_margin)
            # Label cell (no line break)
            pdf.cell(label_width, 6, f"{label}.", ln=0)
            # Text cell (wraps if needed)
            pdf.multi_cell(effective_width - label_width, 6, _safe(ans_text))
        pdf.ln(2)

        # Correct answer line
        correct_idx = q.get("correct_index", None)
        pdf.set_x(pdf.l_margin)
        pdf.set_font("Helvetica", "B", 11)
        if correct_idx is not None and 0 <= correct_idx < len(q["answers"]):
            correct_label = option_labels[correct_idx] if correct_idx < len(option_labels) else f"{correct_idx+1}"
            correct_text = q["answers"][correct_idx]
            correct_line = f"Correct answer: {correct_label}. {correct_text}"
        else:
            correct_line = "Correct answer: (not marked in XML)"
        pdf.multi_cell(effective_width, 6, _safe(correct_line))

        pdf.ln(6)  # space before next question

    pdf.output(pdf_filename)
    return pdf_filename




print(f"PDF generated: {pdf_file}")


PDF generated: Pingo questions with ans in pdf/Predictive_Analytics.pdf


In [42]:
import os

# Directory where all PDFs will go
output_dir = "Pingo questions with ans in pdf"
os.makedirs(output_dir, exist_ok=True)

# Derive a clean base name from the XML file path, e.g.:

xml_base = os.path.splitext(os.path.basename(xml_path))[0]

# Use this as the PDF title (ASCII-only to avoid font issues)
title_from_file = f"{xml_base} -  Questions"

# Full PDF path inside "Pingo questions" directory
pdf_path = os.path.join(output_dir, f"{xml_base}.pdf")

# ---- Call this AFTER you have `questions` from your XML ----
pdf_file = make_pdf_q_and_answer_inline(
    questions,
    pdf_filename=pdf_path,
    title=title_from_file
)

print(f"PDF generated: {pdf_file}")


PDF generated: Pingo questions with ans in pdf/Prescriptive_Analytics.pdf


/var/folders/y2/37476f0n7c793jplk1lghrkjr8cwhw/T/ipykernel_63697/2154952315.py:60: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=0 use new_x=XPos.RIGHT, new_y=YPos.TOP.
  pdf.cell(label_width, 6, f"{label}.", ln=0)
